In [4]:
# 정적으로 메세지를 사용하는 방법

from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage, AIMessage, SystemMessage

model = init_chat_model("google_genai:gemini-2.5-flash-lite")

system_msg = SystemMessage("You are a helpful assistant.")
human_msg = HumanMessage("Hello, how are you?")

message = [system_msg, human_msg]
response = model.invoke(message) # ReturnType: AIMessage
print(response.text)

Hello! I'm doing well, thank you for asking. I'm ready to assist you with any questions or tasks you have. How can I help you today?


In [5]:
# 텍스트 프롬프트
response = model.invoke("Hello, how are you?")

In [7]:
# 메세지 프롬포트

from langchain.messages import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage("You are a poetry expert"),
    HumanMessage("Write a haiku about spring"),
    AIMessage("Cherry blossoms bloom...")
]

response = model.invoke(messages)
print(response.text)


Soft breezes whisper through trees,
Life awakens now.


In [ ]:
# 기본적인 시스템 메세지

system_msg = SystemMessage("You are a helpful coding assistant.")

messages = [
    system_msg,
    HumanMessage("How do I Create a REST API?")
]

response = model.invoke(messages)
print(response.text)

In [11]:
# 자세한 시스템 메세지 페르소나

system_msg = SystemMessage("""
You are a senior Python developer with expertise in web frameworks.
Always provide code examples and explain your reasoning.
Be concise but thorough in your explanations.                           
""")

messages = [
    system_msg,
    HumanMessage("How do I create a REST API?")
]

response = model.invoke(messages)
print(response.text)

Creating a REST API involves designing endpoints, handling requests, and sending responses. The specific implementation depends on the programming language and framework you choose. Here's a breakdown using Python with the Flask framework, a popular and lightweight choice for building web APIs.

### Core Concepts of REST APIs

Before diving into code, let's quickly review REST principles:

*   **Stateless:** Each request from a client to a server must contain all the information needed to understand and process the request. The server should not store any client context between requests.
*   **Client-Server:** A clear separation between the client (user interface) and the server (data storage and logic).
*   **Cacheable:** Responses should indicate whether they can be cached by the client or intermediaries.
*   **Uniform Interface:** A consistent way of interacting with resources, typically using HTTP methods.
    *   **GET:** Retrieve a resource.
    *   **POST:** Create a new resourc

In [ ]:
response = model.invoke([HumanMessage("What is machine learning?")]) # 메세지 객체 생성

response = model.invoke("What is machine learning?") # 단순 문자열 프롬프트

In [ ]:
# 메세지 메타데이터 설정 가능

human_msh = HumanMessage(
    content = "Hello!",
    name="alice", # 사용자 식별자
    id="msg_123", # 메세지 고유 ID
)

In [13]:
# AI 메세지 기능

response = model.invoke("Explain AI")
print(type(response))  # <class 'langchain.messages.ai.AIMessage'>

<class 'langchain_core.messages.ai.AIMessage'>


In [14]:
# AI 메세지 수동 삽입

ai_msg = AIMessage("I'd be happy to help you with that question!")

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("Can you help me?"),
    ai_msg,
    HumanMessage("Great! What's 2+2?")
]

response = model.invoke(messages)
print(response.text)

2 + 2 = 4


In [15]:
# 도구 호출

def get_weather(localtion:str) -> str:
    """Get the weather at a location."""
    
model_with_tools = model.bind_tools([get_weather])
response = model_with_tools.invoke("What's the weather in Paris?")

for tool_call in response.tool_calls:
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")
    print(f"ID: {tool_call['id']}")

Tool: get_weather
Args: {'localtion': 'Paris'}
ID: d82b6b49-0c91-466b-be8a-0efe5c06c9e8


In [16]:
# AIMessage의 토큰 사용 메타데이터

response = model.invoke("Hello!")
response.usage_metadata

{'input_tokens': 3,
 'output_tokens': 11,
 'total_tokens': 14,
 'input_token_details': {'cache_read': 0}}

In [17]:
# 스트리밍을하며 청크를 저장가능

chunks =[]
full_message = None
for chunk in model.stream("Hi"):
    chunks.append(chunk)
    print(chunk.text)
    full_message = chunk if full_message is None else full_message + chunk
    
print("Full message:", full_message.text)


Hi
 there! How can I help you today?

Full message: Hi there! How can I help you today?


In [ ]:
# 도구 메세지
from langchain.messages import ToolMessage

# 모델이 도구 호출
# (여기서는 간결함을 위해 수동으로 메시지를 생성하는 방법)
ai_msg = AIMessage(
    content=[],
    tool_calls=[{
        "name": "get_weather",
        "args": {"location": "San Francisco"},
        "id": "call_123"
    }]
)

# 도구를 실행하고 결과값을 보관
weather_result = "sunny, 72°F"
tool_msg = ToolMessage(
    content=weather_result, 
    tool_call_id="call_123"     # 반드시 Call ID가 일치해야됨
)

messages = [
    HumanMessage("What's the weather in San Francisco?"),
    ai_msg,     # 모델이 도구를 호출
    tool_msg,   # 도구의 응답
]

response = model.invoke(messages)   # 모델이 진행하고 결과를 보관
print(response.text)
print(tool_msg)

The weather in San Francisco is sunny and 72°F.
content='sunny, 72°F' tool_call_id='call_123'


In [ ]:
# 멀티 모달 예시

# content를 제공
human_msg = HumanMessage("Hello, how are you?")

# native 형식
human_msg = HumanMessage(content=[
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image_url", "image_url": {"url": "https://example.com/image.jpg"}}
])

# 일반적인 content block 목록
human_msg = HumanMessage(content_blocks=[
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image", "url": "https://example.com/image.jpg"}
])

In [20]:
# 표준 콘텐츠 블록

message = AIMessage(
    content=[
        {"type": "thinking", "thinking": "...", "signature": "WaUjzkyp.."},
        {"type": "text", "text": "..."}
    ],
    response_metadata={"model_provider": "google"}
)
message.content_blocks

[{'type': 'non_standard',
  'value': {'type': 'thinking', 'thinking': '...', 'signature': 'WaUjzkyp..'}},
 {'type': 'text', 'text': '...'}]

In [ ]:
# 멀티 모달(이미지)

# From URL
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this image."},
        {"type": "text", "url": "https://example.com/path/to/image.jpg"},
    ]
}

# From base64 data
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this image."},
        {
            "type": "image",
            "base64": "AAAAAASDawdknegkjkjsbeb..",
            "mime_type": "image/jpeg",
        },
    ]
}

# From Provider-managed File ID
message={
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this image."},
        {"type": "image", "file_id": "file-abc123"}
    ]
}